In [83]:
import pandas as pd

# Read excel sheets and turn them into lists
instance_dis = pd.read_excel('OR_FP_data.xlsx', '距離')
instance_info = pd.read_excel('OR_FP_data.xlsx', '學校資訊')
instance_school_num = pd.read_excel('OR_FP_data.xlsx', '各里學校數量')
instance_student_num = pd.read_excel('OR_FP_data.xlsx', '各里學生人數')

# distance between 2 villages
distances = instance_dis.iloc[1:31, 2:32]
d = distances.values

# building costs
C_B = instance_info.iloc[0,1]
C_S = instance_info.iloc[1,1]

# capacity of school
N_B = instance_info.iloc[0,3]
N_S = instance_info.iloc[1,3]

# need of teachers
T_B = instance_info.iloc[0,5]
T_S = instance_info.iloc[1,5]

# total number of teacher
K = instance_info.iloc[2,5]

# original school numbers of each village
S_B = instance_school_num.iloc[0:30,1].values
S_S = instance_school_num.iloc[0:30,2].values

# student number of each village
P = instance_student_num.iloc[0:30,2].values

# number of villages
L = range(30)

In [84]:
from gurobipy import *

eg1 = Model("eg1")

x_B = []
for i in L:
    x_B.append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "x_B" + str(i+1)))
x_S = []
for i in L:
    x_S.append(eg1.addVar(lb = 0, vtype = GRB.INTEGER, name = "x_S" + str(i+1)))

z = []
for i in L:
    z.append([])
    for j in L:
        z[i].append(eg1.addVar(lb = 0, vtype = GRB.CONTINUOUS, name="z" + str(i+1) + str(j+1)))

eg1.setObjective(quicksum(quicksum(d[i][j]*z[i][j]*P[i] for j in L) + (x_B[i]-S_B[i])*C_B + (x_S[i]-S_S[i])*C_S for i in L), GRB.MINIMIZE)
eg1.addConstrs((quicksum(z[i][j] for j in L) == 1 for i in L), "i區有多少部分去j區") # correct
eg1.addConstrs(((quicksum(z[i][j]*P[i] for i in L) <= (x_B[j]*N_B + x_S[j]*N_S)) for j in L) ,"需求被滿足")
eg1.addConstr(quicksum((x_B[i]*T_B + x_S[i]*T_S) for i in L) <= K, "可用師資")
eg1.addConstr(quicksum(P[i] for i in L) <= quicksum((x_B[i]*N_B + x_S[i]*N_S) for i in L), "滿足需求")
eg1.addConstrs((x_B[i] - S_B[i] >= 0 for i in L), "不拆掉原學校（大）")
eg1.addConstrs((x_S[i] - S_S[i] >= 0 for i in L), "不拆掉原學校（小）")

{0: <gurobi.Constr *Awaiting Model Update*>,
 1: <gurobi.Constr *Awaiting Model Update*>,
 2: <gurobi.Constr *Awaiting Model Update*>,
 3: <gurobi.Constr *Awaiting Model Update*>,
 4: <gurobi.Constr *Awaiting Model Update*>,
 5: <gurobi.Constr *Awaiting Model Update*>,
 6: <gurobi.Constr *Awaiting Model Update*>,
 7: <gurobi.Constr *Awaiting Model Update*>,
 8: <gurobi.Constr *Awaiting Model Update*>,
 9: <gurobi.Constr *Awaiting Model Update*>,
 10: <gurobi.Constr *Awaiting Model Update*>,
 11: <gurobi.Constr *Awaiting Model Update*>,
 12: <gurobi.Constr *Awaiting Model Update*>,
 13: <gurobi.Constr *Awaiting Model Update*>,
 14: <gurobi.Constr *Awaiting Model Update*>,
 15: <gurobi.Constr *Awaiting Model Update*>,
 16: <gurobi.Constr *Awaiting Model Update*>,
 17: <gurobi.Constr *Awaiting Model Update*>,
 18: <gurobi.Constr *Awaiting Model Update*>,
 19: <gurobi.Constr *Awaiting Model Update*>,
 20: <gurobi.Constr *Awaiting Model Update*>,
 21: <gurobi.Constr *Awaiting Model Update*>

In [85]:
eg1.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 22.3.0 22D68)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 122 rows, 960 columns and 2040 nonzeros
Model fingerprint: 0x0f60f10c
Variable types: 900 continuous, 60 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+03]
  Objective range  [1e+05, 8e+08]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 60 rows and 0 columns
Presolve time: 0.00s
Presolved: 62 rows, 960 columns, 1980 nonzeros
Variable types: 900 continuous, 60 integer (0 binary)
Found heuristic solution: objective 1.610697e+09

Root relaxation: objective 8.072493e+08, 40 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 8.0725e+08    0    8 1.6107e+09 8.0725e+08

In [86]:
print ("\nz* =", eg1.objVal)


z* = 809874097.6083794


In [87]:
for i in L:
    if x_B[i].x-S_B[i] > 0:
        print(x_B[i].varName, '=', x_B[i].x-S_B[i])

x_B23 = 1.0


In [88]:
for i in L:
    if x_S[i].x-S_S[i] > 0:
        print(x_S[i].varName, '=', x_S[i].x-S_S[i])

In [89]:
for i in L:
    for j in L:
        if z[i][j].x != 0:
            print(z[i][j].varName, '=', z[i][j].x)

z18 = 0.8524777340513675
z110 = 0.14752226594863255
z24 = 0.45210500358679984
z28 = 0.5478949964132002
z34 = 1.0
z44 = 1.0
z55 = 1.0
z66 = 1.0
z74 = 1.0
z88 = 1.0
z98 = 0.6937997004357299
z912 = 0.3062002995642701
z1010 = 1.0
z1110 = 1.0
z1212 = 1.0
z1323 = 1.0
z1414 = 1.0
z1519 = 0.39498279326265384
z1524 = 0.12863906463748295
z1525 = 0.47637814209986307
z1614 = 0.8235033990011099
z1624 = 0.17649660099889006
z1712 = 1.0
z1818 = 1.0
z1919 = 1.0
z2023 = 0.8069697815835196
z2024 = 0.1930302184164803
z2121 = 0.4058515423496131
z2123 = 0.5941484576503869
z2210 = 0.25247986972126424
z2218 = 0.7475201302787358
z2323 = 1.0
z2424 = 1.0
z2525 = 1.0
z2619 = 1.0
z275 = 0.987899410683391
z2730 = 0.012100589316608993
z285 = 0.08326824140199213
z2828 = 0.9167317585980078
z294 = 0.09665775401069536
z295 = 0.9033422459893047
z3030 = 1.0


In [90]:
fuel_cost = 0
for i in L:
    for j in L:
        fuel_cost += d[i][j]*z[i][j].x*P[i]
fuel_cost = fuel_cost*E
print(fuel_cost)

39496.390433523076


In [91]:
building_cost = 0
for i in L:
    building_cost += ((x_B[i].x-S_B[i])*C_B +(x_S[i].x-S_S[i])*C_S)
print(building_cost)

800000000.0


In [92]:
total_cost = fuel_cost + building_cost
print(total_cost)

800039496.3904335


In [93]:
# 缺額
v = []
for j in L:
    v.append(x_B[j].x*N_B + x_S[j].x*N_S)
    for i in L:
        v[j] -= z[i][j].x*P[i]
    if v[j] >= 1:
        print('v' + str(j) + '=', v[j])

v5= 622.804084084084
v7= 27.292252252251615
v22= 241.90366366366334
